In [1]:
import pandas as pd
import requests
import numpy as np
import os

import json
from pandas import json_normalize

import findspark
from pyspark.sql import SparkSession

from main import funciones as fun

# solución Querys
## Parte 3

query_3: The transaction_id of an Online purchase on a 31st of December with the highest absolute amount (either earnings or expenses).


Haciendo una revisión rápida en Excel, vemos que son mas de 10 millones de filas. 
Para poder procesar los datos y realizar las querys usaré *Apache Sparck *.



In [2]:
spark=SparkSession.builder.appName("Hackathon_query_3_transactions").getOrCreate()
spark

In [3]:
df_transactions=spark.read.option("inferSchema","true").option("header","true").csv("transactions_data.csv")

In [4]:
df_transactions.count()

13305915

In [5]:
df_transactions.show()

+-------+-------------------+---------+-------+-------+------------------+-----------+---------------+--------------+-------+----+------+
|     id|               date|client_id|card_id| amount|          use_chip|merchant_id|  merchant_city|merchant_state|    zip| mcc|errors|
+-------+-------------------+---------+-------+-------+------------------+-----------+---------------+--------------+-------+----+------+
|7475327|2010-01-01 00:01:00|     1556|   2972|$-77.00| Swipe Transaction|      59935|         Beulah|            ND|58523.0|5499|  NULL|
|7475328|2010-01-01 00:02:00|      561|   4575| $14.57| Swipe Transaction|      67570|     Bettendorf|            IA|52722.0|5311|  NULL|
|7475329|2010-01-01 00:02:00|     1129|    102| $80.00| Swipe Transaction|      27092|          Vista|            CA|92084.0|4829|  NULL|
|7475331|2010-01-01 00:05:00|      430|   2860|$200.00| Swipe Transaction|      27092|    Crown Point|            IN|46307.0|4829|  NULL|
|7475332|2010-01-01 00:06:00|     

In [6]:
df_transactions

DataFrame[id: int, date: timestamp, client_id: int, card_id: int, amount: string, use_chip: string, merchant_id: int, merchant_city: string, merchant_state: string, zip: double, mcc: int, errors: string]

In [22]:
import pyspark.sql.functions as sfunc
from pyspark.sql.functions import max
from pyspark.sql.functions import col
from datetime import  datetime, date
from pyspark.sql.functions import month
from pyspark.sql.functions import day

In [14]:
df_min_max_date=df_transactions.select(sfunc.min("date"), sfunc.max("date"))
df_min_max_date.show()

+-------------------+-------------------+
|          min(date)|          max(date)|
+-------------------+-------------------+
|2010-01-01 00:01:00|2019-10-31 23:59:00|
+-------------------+-------------------+



In [25]:
df1=df_transactions.withColumn("month_transc", month(df_transactions.date))
df1.show()

+-------+-------------------+---------+-------+-------+------------------+-----------+---------------+--------------+-------+----+------+------------+
|     id|               date|client_id|card_id| amount|          use_chip|merchant_id|  merchant_city|merchant_state|    zip| mcc|errors|month_transc|
+-------+-------------------+---------+-------+-------+------------------+-----------+---------------+--------------+-------+----+------+------------+
|7475327|2010-01-01 00:01:00|     1556|   2972|$-77.00| Swipe Transaction|      59935|         Beulah|            ND|58523.0|5499|  NULL|           1|
|7475328|2010-01-01 00:02:00|      561|   4575| $14.57| Swipe Transaction|      67570|     Bettendorf|            IA|52722.0|5311|  NULL|           1|
|7475329|2010-01-01 00:02:00|     1129|    102| $80.00| Swipe Transaction|      27092|          Vista|            CA|92084.0|4829|  NULL|           1|
|7475331|2010-01-01 00:05:00|      430|   2860|$200.00| Swipe Transaction|      27092|    Crow

In [27]:
df_diciembre=df1.filter(df1.month_transc == 12)

In [28]:
df1=df_diciembre.withColumn("day_transc", day(df_transactions.date))
df1.show()

+-------+-------------------+---------+-------+-------+------------------+-----------+---------------+--------------+-------+----+------+------------+----------+
|     id|               date|client_id|card_id| amount|          use_chip|merchant_id|  merchant_city|merchant_state|    zip| mcc|errors|month_transc|day_transc|
+-------+-------------------+---------+-------+-------+------------------+-----------+---------------+--------------+-------+----+------+------------+----------+
|8835666|2010-12-01 00:01:00|      641|     14| $63.17| Swipe Transaction|      26549|       Columbus|            OH|43228.0|5912|  NULL|          12|         1|
|8835667|2010-12-01 00:03:00|      530|   5110|  $9.80| Swipe Transaction|      50783|    Bloomington|            CA|92316.0|5411|  NULL|          12|         1|
|8835668|2010-12-01 00:03:00|     1684|     94| $37.28|Online Transaction|      39021|         ONLINE|          NULL|   NULL|4784|  NULL|          12|         1|
|8835669|2010-12-01 00:07:00

In [33]:
df_diciembre_31=df1.filter((df1.month_transc == 12) & (df1.day_transc == 31))
df_diciembre.show()

+-------+-------------------+---------+-------+-------+------------------+-----------+-----------------+--------------+-------+----+-------+------------+----------+
|     id|               date|client_id|card_id| amount|          use_chip|merchant_id|    merchant_city|merchant_state|    zip| mcc| errors|month_transc|day_transc|
+-------+-------------------+---------+-------+-------+------------------+-----------+-----------------+--------------+-------+----+-------+------------+----------+
|8962673|2010-12-31 00:01:00|     1684|     94| $23.16|Online Transaction|      39021|           ONLINE|          NULL|   NULL|4784|   NULL|          12|        31|
|8962676|2010-12-31 00:05:00|      641|   4718| $90.01| Swipe Transaction|       4312|          Chicago|            IL|60639.0|5411|   NULL|          12|        31|
|8962677|2010-12-31 00:06:00|      848|   3915| $60.28| Swipe Transaction|      13051|          Harwood|            MD|20776.0|5813|   NULL|          12|        31|
|8962678|2

In [34]:
df_diciembre_31.count()

30776

In [35]:
df_diciembre_31_transc=df_diciembre_31.toPandas()

In [54]:
spark.stop()

In [38]:
fun.eda_basico(df_diciembre_31_transc)

Muestra de datos:


,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors,month_transc,day_transc
27919,22322922,2018-12-31 07:54:00,1435,4840,$4.66,Chip Transaction,9254,Alpharetta,GA,30004.0,5812,None,12,31
14007,15467739,2014-12-31 08:31:00,879,4500,$50.94,Swipe Transaction,27681,Brownsville,TX,78521.0,7230,None,12,31


Columnas: 14
Filas: 30776
Valores duplicados:0
Valores nulos encontrados:
id                    0
date                  0
client_id             0
card_id               0
amount                0
use_chip              0
merchant_id           0
merchant_city         0
merchant_state     3737
zip                3968
mcc                   0
errors            30269
month_transc          0
day_transc            0
dtype: int64
Información general:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30776 entries, 0 to 30775
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              30776 non-null  int32         
 1   date            30776 non-null  datetime64[ns]
 2   client_id       30776 non-null  int32         
 3   card_id         30776 non-null  int32         
 4   amount          30776 non-null  object        
 5   use_chip        30776 non-null  object        
 6   merchant_id     30776 non-

In [44]:
df_diciembre_31_transc["amount"]=df_diciembre_31_transc["amount"].str.replace("$","")

In [45]:
df_diciembre_31_transc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30776 entries, 0 to 30775
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              30776 non-null  int32         
 1   date            30776 non-null  datetime64[ns]
 2   client_id       30776 non-null  int32         
 3   card_id         30776 non-null  int32         
 4   amount          30776 non-null  object        
 5   use_chip        30776 non-null  object        
 6   merchant_id     30776 non-null  int32         
 7   merchant_city   30776 non-null  object        
 8   merchant_state  27039 non-null  object        
 9   zip             26808 non-null  float64       
 10  mcc             30776 non-null  int32         
 11  errors          507 non-null    object        
 12  month_transc    30776 non-null  int32         
 13  day_transc      30776 non-null  int32         
dtypes: datetime64[ns](1), float64(1), int32(7), object(5)


In [46]:
df_diciembre_31_transc["amount"]=df_diciembre_31_transc["amount"].astype(float)

In [47]:
df_diciembre_31_transc["amount"].dtype

dtype('float64')

In [48]:
df_diciembre_31_transc.sample(5)

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors,month_transc,day_transc
26455,20603749,2017-12-31 16:44:00,1202,2885,8.17,Online Transaction,9932,ONLINE,None,NaN,5311,None,12,31
19459,17171593,2015-12-31 17:13:00,823,5524,16.54,Chip Transaction,15961,Suffolk,VA,23434.0,7832,None,12,31
12345,13797689,2013-12-31 16:22:00,1682,3407,11.30,Swipe Transaction,81833,Grand Forks,ND,58203.0,5912,None,12,31
6093,10536767,2011-12-31 20:44:00,323,5977,13.04,Swipe Transaction,26810,Hilmar,CA,95324.0,5541,None,12,31
24954,20601899,2017-12-31 10:38:00,756,3963,117.03,Chip Transaction,27601,Bellbrook,OH,45305.0,7538,None,12,31


In [50]:
df_diciembre_31_transc.describe().T

,count,mean,min,25%,50%,75%,max,std
id,30776.0,15656303.454283,8962673.0,12145299.75,15469454.5,18880611.25,22326461.0,4283793.530683
date,30776,2015-01-25 15:23:20.332077056,2010-12-31 00:01:00,2012-12-31 11:36:00,2014-12-31 14:35:00,2016-12-31 17:56:15,2018-12-31 23:58:00,NaN
client_id,30776.0,1028.375195,0.0,523.0,1072.0,1535.0,1998.0,582.71323
card_id,30776.0,3497.891311,0.0,2437.75,3628.5,4933.25,6137.0,1664.940711
amount,30776.0,43.669491,-498.0,11.18,31.61,65.4825,1184.71,71.394917
merchant_id,30776.0,47845.746101,22.0,26232.0,45926.0,68689.25,100340.0,26013.177227
zip,26808.0,52121.941249,1202.0,28801.0,48146.0,78520.0,99504.0,29375.691884
mcc,30776.0,5605.242462,1711.0,5300.0,5499.0,5813.0,9402.0,920.361762
month_transc,30776.0,12.0,12.0,12.0,12.0,12.0,12.0,0.0
day_transc,30776.0,31.0,31.0,31.0,31.0,31.0,31.0,0.0


In [51]:
cond_1=df_diciembre_31_transc["amount"]==df_diciembre_31_transc["amount"].min()
df_cond1=df_diciembre_31_transc[cond_1]

In [52]:
df_cond1

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors,month_transc,day_transc
9461,12147446,2012-12-31 20:11:00,1781,3032,-498.0,Swipe Transaction,3558,Goshen,IN,46526.0,3640,None,12,31
23698,18881342,2016-12-31 22:24:00,631,2596,-498.0,Swipe Transaction,57133,Waianae,HI,96792.0,3730,None,12,31
27342,20604808,2017-12-31 23:23:00,1241,3501,-498.0,Chip Transaction,59474,Brookville,OH,45309.0,3722,None,12,31


In [65]:
df_query=pd.DataFrame(df_cond1["id"])


In [66]:
df_query.to_csv("query_3",index=False)

In [67]:
js_query=df_query.to_json(orient="columns", index=None)
js_query

'{"id":{"9461":12147446,"23698":18881342,"27342":20604808}}'

# Parte 3
## Query 4:

query_4: Which client over the age of 40 made the most transactions with a Visa card in February 2016? Please return the client_id, the card_id involved, and the total number of transactions.


query_4: ¿Qué cliente mayor de 40 años realizó el mayor número de transacciones con una tarjeta Visa en febrero de 2016? Por favor, proporciona el client_id, el card_id involucrado, y el número total de transacciones.



In [37]:
spark=SparkSession.builder.appName("Hackathon_query_4_transactions").getOrCreate()
spark

In [3]:
df_transactions=spark.read.option("inferSchema","true").option("header","true").csv("transactions_data.csv")

In [70]:
df_transactions.show()

+-------+-------------------+---------+-------+-------+------------------+-----------+---------------+--------------+-------+----+------+
|     id|               date|client_id|card_id| amount|          use_chip|merchant_id|  merchant_city|merchant_state|    zip| mcc|errors|
+-------+-------------------+---------+-------+-------+------------------+-----------+---------------+--------------+-------+----+------+
|7475327|2010-01-01 00:01:00|     1556|   2972|$-77.00| Swipe Transaction|      59935|         Beulah|            ND|58523.0|5499|  NULL|
|7475328|2010-01-01 00:02:00|      561|   4575| $14.57| Swipe Transaction|      67570|     Bettendorf|            IA|52722.0|5311|  NULL|
|7475329|2010-01-01 00:02:00|     1129|    102| $80.00| Swipe Transaction|      27092|          Vista|            CA|92084.0|4829|  NULL|
|7475331|2010-01-01 00:05:00|      430|   2860|$200.00| Swipe Transaction|      27092|    Crown Point|            IN|46307.0|4829|  NULL|
|7475332|2010-01-01 00:06:00|     

In [4]:
from pyspark.sql.functions import month , dayofmonth

In [18]:
df_trans_feb_16=df_transactions.filter((month(df_transactions.date)== 2) & (dayofmonth(df_transactions.date)== 16))
df_trans_feb_16.show()

+-------+-------------------+---------+-------+-------+------------------+-----------+-----------------+--------------+-------+----+------+
|     id|               date|client_id|card_id| amount|          use_chip|merchant_id|    merchant_city|merchant_state|    zip| mcc|errors|
+-------+-------------------+---------+-------+-------+------------------+-----------+-----------------+--------------+-------+----+------+
|7656236|2010-02-16 00:01:00|      376|   1262| $39.81| Swipe Transaction|       8255|       Vincentown|            NJ| 8088.0|5813|  NULL|
|7656237|2010-02-16 00:01:00|      554|   2424|  $3.08| Swipe Transaction|      13523|         La Porte|            TX|77571.0|5310|  NULL|
|7656238|2010-02-16 00:01:00|      992|   5007|$120.00| Swipe Transaction|      27092|       South Hill|            VA|23970.0|4829|  NULL|
|7656239|2010-02-16 00:01:00|     1797|   1127| $47.99| Swipe Transaction|      54343|      San Leandro|            CA|94577.0|4121|  NULL|
|7656240|2010-02-16 

In [19]:
df_trans_feb_16.count()

37674

In [20]:
df_trans_feb_16=df_trans_feb_16.toPandas()

In [22]:
df_trans_feb_16_copy=df_trans_feb_16.copy()

In [23]:
fun.eda_basico(df_trans_feb_16_copy)

Muestra de datos:


,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
5792,9163702,2011-02-16 15:09:00,619,4519,$60.33,Swipe Transaction,59935,Rochester,MN,55901.0,5499,None
22745,17388334,2016-02-16 07:19:00,802,5676,$80.36,Chip Transaction,32175,Penryn,CA,95663.0,7538,None


Columnas: 12
Filas: 37674
Valores duplicados:0
Valores nulos encontrados:
id                    0
date                  0
client_id             0
card_id               0
amount                0
use_chip              0
merchant_id           0
merchant_city         0
merchant_state     4407
zip                4619
mcc                   0
errors            37118
dtype: int64
Información general:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37674 entries, 0 to 37673
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              37674 non-null  int32         
 1   date            37674 non-null  datetime64[ns]
 2   client_id       37674 non-null  int32         
 3   card_id         37674 non-null  int32         
 4   amount          37674 non-null  object        
 5   use_chip        37674 non-null  object        
 6   merchant_id     37674 non-null  int32         
 7   merchant_city   37674 

In [26]:
def limpieza_dollar(df,columna):
    df[columna]=df[columna].str.replace("$","")
    df[columna]=df[columna].astype(float)
    return df

In [27]:
df_trans_feb_16=limpieza_dollar(df_trans_feb_16, "amount")

In [28]:
df_trans_feb_16.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37674 entries, 0 to 37673
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              37674 non-null  int32         
 1   date            37674 non-null  datetime64[ns]
 2   client_id       37674 non-null  int32         
 3   card_id         37674 non-null  int32         
 4   amount          37674 non-null  float64       
 5   use_chip        37674 non-null  object        
 6   merchant_id     37674 non-null  int32         
 7   merchant_city   37674 non-null  object        
 8   merchant_state  33267 non-null  object        
 9   zip             33055 non-null  float64       
 10  mcc             37674 non-null  int32         
 11  errors          556 non-null    object        
dtypes: datetime64[ns](1), float64(2), int32(5), object(4)
memory usage: 2.7+ MB


In [29]:
df_trans_feb_16_copy.rename(columns={"id":"transaction_id"}, inplace=True)

In [30]:
df_data_client_card_merge=pd.read_csv("users_card_final.csv")

In [33]:
df_trans_16_feb_merge=df_trans_feb_16_copy.merge(df_data_client_card_merge, left_on="client_id",right_on="client_id")

In [35]:
df_trans_16_feb_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144509 entries, 0 to 144508
Data columns (total 41 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   transaction_id         144509 non-null  int32         
 1   date                   144509 non-null  datetime64[ns]
 2   client_id              144509 non-null  int32         
 3   card_id_x              144509 non-null  int32         
 4   amount                 144509 non-null  object        
 5   use_chip               144509 non-null  object        
 6   merchant_id            144509 non-null  int32         
 7   merchant_city          144509 non-null  object        
 8   merchant_state         127723 non-null  object        
 9   zip                    126955 non-null  float64       
 10  mcc                    144509 non-null  int32         
 11  errors                 2114 non-null    object        
 12  current_age            144509 non-null  int6

In [36]:
df_trans_16_feb_merge.drop(["card_id_x"], axis="columns",inplace=True)

In [38]:
df_trans_16_feb_merge.to_csv("transactions_client_cards_16_febrero.csv")

In [40]:
df_transactions_16=spark.read.option("inferSchema","true").option("header","true").csv("transactions_client_cards_16_febrero.csv")

In [41]:
df_transactions_16.show()

+---+--------------+-------------------+---------+-------+------------------+-----------+-------------+--------------+-------+----+------+-----------+--------------+----------+-----------+------+--------------------+--------+---------+-----------------+-------------+----------+------------+----------------+---------+----------+---------------+----------------+-------+---+--------+----------------+------------+--------------+---------------------+----------------+--------------------+-------------------+-------------+------------+
|_c0|transaction_id|               date|client_id| amount|          use_chip|merchant_id|merchant_city|merchant_state|    zip| mcc|errors|current_age|retirement_age|birth_year|birth_month|gender|             address|latitude|longitude|per_capita_income|yearly_income|total_debt|credit_score|num_credit_cards|card_id_y|card_brand|      card_type|     card_number|expires|cvv|has_chip|num_cards_issued|credit_limit|acct_open_date|year_pin_last_changed|card_on_dark_w

In [58]:
df_40=df_transactions_16.filter(df_transactions_16.current_age == 40).show()

+---+--------------+-------------------+---------+-------+-----------------+-----------+--------------+--------------+-------+----+--------------------+-----------+--------------+----------+-----------+------+--------------------+--------+---------+-----------------+-------------+----------+------------+----------------+---------+----------+---------------+----------------+-------+---+--------+----------------+------------+--------------+---------------------+----------------+--------------------+-------------------+-------------+------------+
|_c0|transaction_id|               date|client_id| amount|         use_chip|merchant_id| merchant_city|merchant_state|    zip| mcc|              errors|current_age|retirement_age|birth_year|birth_month|gender|             address|latitude|longitude|per_capita_income|yearly_income|total_debt|credit_score|num_credit_cards|card_id_y|card_brand|      card_type|     card_number|expires|cvv|has_chip|num_cards_issued|credit_limit|acct_open_date|year_pin

In [42]:
df_transactions_16.createOrReplaceTempView("table_trans")

In [90]:
consulta=""" SELECT client_id, card_id_y, count(transaction_id) AS total_transacciones
FROM table_trans
WHERE current_age = 40
GROUP BY client_id, card_id_y
ORDER BY total_transacciones DESC
LIMIT 1
"""

In [91]:
df_query_4=spark.sql(consulta)
df_query_4.show()

+---------+---------+-------------------+
|client_id|card_id_y|total_transacciones|
+---------+---------+-------------------+
|      190|     3363|                 74|
+---------+---------+-------------------+



In [93]:
df_query_4.columns

['client_id', 'card_id_y', 'total_transacciones']

In [94]:
def final_query(df, query):
    df_query=df.toPandas()
    df_query.to_csv(query,index=False)
    js_query=df_query.to_json(orient="records")
    return js_query

In [95]:

final_query(df_query_4, "query_4")

'[{"client_id":190,"card_id_y":3363,"total_transacciones":74}]'